In [ ]:
import teetool as tt
import pandas as pd

In [ ]:
# create a world and add existing data

# new world -- nres defines default resolution of tube/logp grid generation
world = tt.World(name="crs31", ndim=3)

list_filename = ("1_standard_launch", "2_Southern_wind", "3_parachute_failure", "4_thrust_variation")

for filename in list_filename:
    # extract data from pickle
    cluster_data = pd.read_pickle("temp/{0}.pickle".format(filename));
    
    # add data to the world
    world.addCluster(cluster_data, filename)

In [ ]:
world.buildModel(settings={"model_type":"resampling", "ngaus":100})
world.overview()

In [ ]:
# compare statistics for different resolutions

print("resolution 100")
world.getTubeStats(sdwidth=1, resolution=100)

print("resolution 50")
world.getTubeStats(sdwidth=1, resolution=50)

print("resolution 20")
world.getTubeStats(sdwidth=1, resolution=20)

In [ ]:
# visualise 2d -- trend & diff(erence)

% matplotlib inline

for kind in ["trend"]:
    for i, name in enumerate(list_filename[:]):
        for z in range(0, 3000, 50):
            visual = tt.visual_2d.Visual_2d(world, figsize=(5, 5), dpi=600, tight_layout=True)
            if kind == "trend":
                visual.plotTube(list_icluster=[i], z=z, resolution=[100, 100], colour=(1.0, 0.3, 0.3))
                save_as = "{0}_{1}".format(name, z)
            elif kind == "diff":
                visual.plotTubeDifference([0, i], z=z, resolution=[100, 100])
                save_as = "diff_0_{0}_{1}".format(i, z)
            # plot mean
            visual.plotMean(list_icluster=[i], colour=(0.0, 0.0, 0.0), linestyle='--')
            # set labels
            visual._ax.set_xlabel("Easting [m]")
            visual._ax.set_ylabel("Northing [m]")
            visual._ax.set_xlim([-1000, 1000])
            visual._ax.set_ylim([0, 3000])
            visual._ax.set_title("")
            # equal scale
            visual._ax.set_aspect('equal', adjustable='box')
            # store image
            visual.save(add=save_as)
            # close figure
            visual.close()

In [ ]:
% matplotlib inline

for kind in ["diff"]:
    for i, name in enumerate(list_filename[:]):
        for z in range(0, 3000, 50):
            visual = tt.visual_2d.Visual_2d(world, figsize=(5, 5), dpi=600, tight_layout=True)
            if kind == "trend":
                visual.plotTube(list_icluster=[i], z=z, resolution=[100, 100], colour=(1.0, 0.3, 0.3))
                save_as = "{0}_{1}".format(name, z)
            elif kind == "diff":
                visual.plotTubeDifference([0, i], z=z, resolution=[100, 100])
                save_as = "diff_0_{0}_{1}".format(i, z)
            # plot mean
            visual.plotMean(list_icluster=[0, i], colour=(0.0, 0.0, 0.0), linestyle='--')
            # set labels
            visual._ax.set_xlabel("Easting [m]")
            visual._ax.set_ylabel("Northing [m]")
            visual._ax.set_xlim([-1000, 1000])
            visual._ax.set_ylim([0, 3000])
            visual._ax.set_title("")
            # equal scale
            visual._ax.set_aspect('equal', adjustable='box')
            # store image
            visual.save(add=save_as)
            # close figure
            visual.close()

In [ ]:
# visualise 3d -- 50 trajectories and trend

for kind in ["trend", "diff"]:
    for i, name in enumerate(list_filename[:]):
        visual = tt.visual_3d.Visual_3d(world, size=(1200, 900))
        if kind == "trend":
            # visualise trajectories
            visual.plotTrajectories([i], ntraj=50, colour=(0.0,0.0,0.0))
            # visualise tube
            visual.plotTube([i], alpha=0.5, resolution=20, colour=(1.0, 0.3, 0.3))
            # save as string
            save_as = "{0}".format(name)
        if kind == "diff":
            # visualise difference
            visual.plotTubeDifference([0, i], alpha=1.0, resolution=20)
            # save as string
            save_as = "diff_0_{0}".format(i)
        
        # 1000m = 1km = 1 km^3 is one grid block
        visual.plotGrid(resolution=500,
                        outline=[-1000, 1000, 0, 3000, 0, 3000])
        # labels and ticks
        visual.setLabels("Easting [m]", "Northing [m]", "Altitude [m]")
        visual.setAxesFormat("%.0f")  # no ticks
        
        # modify azimuth
        for azi in range(0, 100, 5):
            # change view
            visual.setView(azimuth=azi, elevation=87, focalpoint=[0, 1400, 1300], distance=9200)
            # store image
            save_as_azi = "{0}_{1}".format(save_as, azi)
            visual.save(add=save_as_azi)
        

visual.close() # use visual.show() to pause figures at end and not auto-close